In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import jax.random as r
from jax.random import key as prngkey
import jax.numpy as jnp
from condorgmm.condor.types import Intrinsics, Pose
from condorgmm.condor.geometry import xyz_to_cameraxyd, xyz_from_cameraxyd, find_aligning_pose
import pytest

In [ ]:
def _generate_zero_error_pose_alignment_problem(key):
    k1, k2, k3, k4, k5 = r.split(key, 5)
    xyz_0 = r.uniform(k1, shape=(100, 3), minval=-10.0, maxval=10.0)
    pose_position = r.uniform(k2, shape=(3,), minval=-10.0, maxval=10.0)
    quat = r.normal(k3, shape=(4,))
    quat = quat / jnp.linalg.norm(quat)
    pose = Pose(position=pose_position, quaternion=quat)
    xyz_1 = pose.apply(xyz_0)
    
    # randomly mask out 20%
    mask = r.bernoulli(k4, p=1.0, shape=(100,))
    xyz_1 = jnp.where(
        mask[:, None],
        xyz_1,
        r.uniform(k5, shape=(100, 3), minval=-10.0, maxval=10.0)
    )

    return xyz_0, xyz_1, mask, pose


In [ ]:
key = r.key(0)
xyz_0, xyz_1, mask, pose = _generate_zero_error_pose_alignment_problem(key)

In [ ]:
inferred_pose = find_aligning_pose(xyz_0, xyz_1, mask)
print(pose)
print(inferred_pose)

In [ ]:
jnp.linalg.norm(inferred_pose.apply(xyz_0) - xyz_1)